In [1]:
import os
import csv
import math

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
import os
import csv
import math
import itertools
import pickle

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import os
import csv
import math
import itertools
import pickle

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from scipy import sparse
import networkx as nx

from mbi import (
    Dataset,
    FactoredInference,
    Domain,
    LocalInference,
    MixtureInference,
    PublicInference,
)
import os
import csv
import math
import itertools
import pickle

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from fairness_checker import *

In [2]:
# source: https://www.kaggle.com/datasets/uciml/adult-census-income/data
dataset = pd.read_csv("./data/adult_og.csv")
dataset.head()

# Convert Sex value to 0 and 1
dataset["sex"] = dataset["sex"].map({"Male": 0, "Female":1})

# Create Married Column - Binary Yes(1) or No(0)
dataset["marital.status"] = dataset["marital.status"].replace(['Never-married','Divorced','Separated','Widowed'], 'Single')
dataset["marital.status"] = dataset["marital.status"].replace(['Married-civ-spouse','Married-spouse-absent','Married-AF-spouse'], 'Married')
dataset["marital.status"] = dataset["marital.status"].map({"Married":1, "Single":0})
dataset["marital.status"] = dataset["marital.status"].astype(int)
dataset['income'] = dataset['income'].map({'<=50K': 0, '>50K': 1})

columns_to_encode = ['age', 'workclass', 'fnlwgt', 'education', 'education.num', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country', 'capital.gain', 'capital.loss', 'hours.per.week']
le = preprocessing.LabelEncoder()
for column in columns_to_encode:
    dataset[column] = le.fit_transform(dataset[column])

dataset.head()

# Drop the data you don't want to use
dataset_small = dataset.copy()
dataset_small.drop(labels=["workclass","education","occupation","relationship","race","native.country"], axis = 1, inplace = True)
dataset_small.drop(labels=["fnlwgt","capital.gain","capital.loss"], axis = 1, inplace = True)

dataset_small.to_csv('./data/adult_processed_small.csv', index=False)
dataset_small.head()

,age,education.num,marital.status,sex,hours.per.week,income
0,72,8,0,1,39,0
1,65,8,0,1,17,0
2,49,9,0,1,39,0
3,37,3,0,1,39,0
4,24,9,0,1,39,0


In [7]:
dataset_small = pd.read_csv("./data/adult_processed_small.csv")

X_small = dataset_small[["age", "education.num", "marital.status", "sex", "hours.per.week"]].copy()
Y_small = dataset_small['income']

X_train_small, X_test_small, Y_train_small, Y_test_small = train_test_split(X_small, Y_small, test_size=0.2, random_state=7)

clf_small = RandomForestClassifier(
        n_estimators=250,
        max_features=4
    )

clf_small.fit(X_train_small, Y_train_small)

test_accuracy = clf_small.score(X_test_small, Y_test_small)
print(f"Test Accuracy: {test_accuracy}")

with open('./model/adult_RF_small.pkl', 'wb') as f:
    pickle.dump(clf_small, f)

Test Accuracy: 0.8082296944572394


In [3]:
data_small = Dataset.load("./data/adult_processed_small.csv", "./data/adult_processed_small.json")
domain_small = data_small.domain
total = data_small.df.shape[0]

# fully connected graph
fields = list(data_small.domain)
cliques = [
    (fields[i], fields[j]) for i in range(len(fields)) for j in range(i+1, len(fields))
]

# spend half of privacy budget to measure all 1 way marginals
np.random.seed(0)

epsilon = 1.0
epsilon_split = epsilon / (len(data_small.domain) + len(cliques))
sigma = 2.0 / epsilon_split

measurements = []
for col in data_small.domain:
    x = data_small.project(col).datavector()
    y = x + np.random.laplace(loc=0, scale=sigma, size=x.size)
    I = sparse.eye(x.size)
    measurements.append((I, y, sigma, (col,)))

# spend half of privacy budget to measure some more 2 and 3 way marginals
for cl in cliques:
    x = data_small.project(cl).datavector()
    y = x + np.random.laplace(loc=0, scale=sigma, size=x.size)
    I = sparse.eye(x.size)
    measurements.append((I, y, sigma, cl))

engine = FactoredInference(domain_small, log=True, iters=500)
model = engine.estimate(measurements, total=total)

# Save the model to a file
with open('./model/adult_synth_small.pkl', 'wb') as f:
    pickle.dump(model, f)

Total clique size: 878336
iteration		time		l1_loss		l2_loss		feasibility
0.00		0.00		23239.12		473062.72		0.00
50.00		10.62		11926.47		13136.75		0.00
100.00		25.41		10931.10		10907.11		0.00
150.00		39.82		10671.36		10469.35		0.00
200.00		54.02		10519.53		10229.52		0.00
250.00		67.26		10423.80		10063.28		0.00
300.00		80.61		10345.12		9929.22		0.00
350.00		95.15		10290.00		9821.02		0.00
400.00		109.87		10240.06		9725.26		0.00
450.00		124.62		10200.80		9646.04		0.00


In [12]:
# To load the model back
with open('./model/adult_RF_small.pkl', 'rb') as f:
    clf = pickle.load(f)
with open('./model/adult_synth_small.pkl', 'rb') as f:
    model = pickle.load(f)

class model_wrapper:
    def __init__(self, model):
        self.model = model

    def predict(self, file):
        df = pd.read_csv(file)
        X = self.preprocessing(df)

        return self.model.predict(X)

    def preprocessing(self, df):
        df = df[["age", "education.num", "marital.status", "sex", "hours.per.week"]].copy()

        return df

trained = model_wrapper(clf)

iter = 10

disparate_impact = []
demographic_parity = []
equalized_odds_1, equalized_odds_2 = [], []
equal_opportunity = []
accuracy_eqaulity = []
predictive_parity = []
equal_calibration = []
conditional_statistical_parity = []
predictive_equality = []
conditional_use_accuracy_equality_1, conditional_use_accuracy_equality_2 = [], []
positive_balance = []
negative_balance = []
mean_difference = []

for _ in range(iter):
    synth = model.synthetic_data(rows=30000)
    sdf = synth.df
    sdf.to_csv('./tmp/synth.csv', index=False)

    c = fairness_model_checker("./tmp/synth.csv", verbose=False)

    privileged_predicate = lambda row: row['sex'] != '0'
    positive_predicate = lambda Y: Y == 1
    truth_predicate = lambda row: row['income'] == '1'

    disparate_impact               .append( c.disparate_impact(0.8, trained, privileged_predicate, positive_predicate, value=True) )
    demographic_parity             .append( c.demographic_parity(0.2, trained, privileged_predicate, positive_predicate, value=True) )
    (x1, x2) = c.equalized_odds(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    equalized_odds_1 .append( x1 )
    equalized_odds_2 .append( x2 )
    equal_opportunity              .append( c.equal_opportunity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    accuracy_eqaulity              .append( c.accuracy_eqaulity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    predictive_parity              .append( c.predictive_parity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    # equal_calibration (n/a)
    conditional_statistical_parity .append( c.conditional_statistical_parity(0.2, trained, privileged_predicate, positive_predicate, lambda x: (lambda row: row['marital.status'] == x), ('0',), value=True) )
    predictive_equality            .append( c.predictive_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    (y1, y2) = c.conditional_use_accuracy_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    conditional_use_accuracy_equality_1 .append( y1 )
    conditional_use_accuracy_equality_2 .append( y2 )
    # positive_balance (n/a)
    # negative_balance (n/a)
    mean_difference .append( c.mean_difference(0.2,trained, privileged_predicate, positive_predicate, value=True) )

og_disparate_impact = 0
og_demographic_parity = 0
og_equalized_odds_1, og_equalized_odds_2 = 0, 0
og_equal_opportunity = 0
og_accuracy_eqaulity = 0
og_predictive_parity = 0
og_equal_calibration = 0
og_conditional_statistical_parity = 0
og_predictive_equality = 0
og_conditional_use_accuracy_equality_1, og_conditional_use_accuracy_equality_2 = 0, 0
og_positive_balance = 0
og_negative_balance = 0
og_mean_difference = 0

cog = fairness_model_checker("./data/adult_processed_small.csv", verbose=False)

privileged_predicate = lambda row: row['sex'] == '0'
positive_predicate = lambda Y: Y == 1
truth_predicate = lambda row: row['income'] == '1'

og_disparate_impact = cog.disparate_impact(0.8, trained, privileged_predicate, positive_predicate, value=True)
og_demographic_parity = cog.demographic_parity(0.2, trained, privileged_predicate, positive_predicate, value=True)
(og_equalized_odds_1, og_equalized_odds_2) = cog.equalized_odds(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_equal_opportunity = cog.equal_opportunity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_accuracy_eqaulity = cog.accuracy_eqaulity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_predictive_parity = cog.predictive_parity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
# equal_calibration (n/a)
og_conditional_statistical_parity = cog.conditional_statistical_parity(0.2, trained, privileged_predicate, positive_predicate, lambda x: (lambda row: row['marital.status'] == x), ('0',), value=True)
og_predictive_equality = cog.predictive_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
(og_conditional_use_accuracy_equality_1, og_conditional_use_accuracy_equality_2) = cog.conditional_use_accuracy_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
# positive_balance (n/a)
# negative_balance (n/a)
og_mean_difference = cog.mean_difference(0.2, trained, privileged_predicate, positive_predicate, value=True)

print("name               ", "og          ", "synth       ", "diff        ", "std")
print("demographic_parity ", f"{og_demographic_parity:.10f}", f"{np.mean(np.array(demographic_parity)):.10f}", f"{abs(og_demographic_parity - np.mean(np.array(demographic_parity))):.10f}", np.std(np.array(demographic_parity)))
print("accuracy_eqaulity  ", f"{og_accuracy_eqaulity:.10f}", f"{np.mean(np.array(accuracy_eqaulity)):.10f}", f"{abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity))):.10f}", np.std(np.array(accuracy_eqaulity)))
print("equalized_odds_1   ", f"{og_equalized_odds_1:.10f}", f"{np.mean(np.array(equalized_odds_1)):.10f}", f"{abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1))):.10f}", np.std(np.array(equalized_odds_1)))
print("equalized_odds_2   ", f"{og_equalized_odds_2:.10f}", f"{np.mean(np.array(equalized_odds_2)):.10f}", f"{abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2))):.10f}", np.std(np.array(equalized_odds_2)))
print("accuracy_equality_1", f"{og_conditional_use_accuracy_equality_1:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_1)):.10f}", f"{abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1))):.10f}", np.std(np.array(conditional_use_accuracy_equality_1)))
print("accuracy_equality_2", f"{og_conditional_use_accuracy_equality_2:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_2)):.10f}", f"{abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2))):.10f}", np.std(np.array(conditional_use_accuracy_equality_2)))
print("mean_difference    ", f"{og_mean_difference:.10f}", f"{np.mean(np.array(mean_difference)):.10f}", f"{abs(og_mean_difference - np.mean(np.array(mean_difference))):.10f}", np.std(np.array(mean_difference)))

print(
abs(og_demographic_parity - np.mean(np.array(demographic_parity)))+
abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity)))+
abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1)))+
abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2)))+
abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1)))+
abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2)))+
abs(og_mean_difference - np.mean(np.array(mean_difference)))
)

name                og           synth        diff         std
demographic_parity  0.1794590492 0.1644071458 0.0150519034 0.0015610252491575218
accuracy_eqaulity   0.1073618804 0.1186453820 0.0112835016 0.001860403129945866
equalized_odds_1    0.0709861889 0.0760730500 0.0050868610 0.0018632981056573932
equalized_odds_2    0.0114556016 0.1594259566 0.1479703550 0.006261589742373502
accuracy_equality_1 0.1956401683 0.1986465833 0.0030064150 0.014396901512254407
accuracy_equality_2 0.1256246141 0.2012349033 0.0756102893 0.004239971965642017
mean_difference     0.1635085994 0.1411724776 0.0223361218 0.002030475289693193
0.28034544711468884
